In [1]:
import numpy as np
import pandas as pd
import pyaldata as pyal
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler

import sys
sys.path.append('../src')
import prep # make sure to have pyaldata installed

In [2]:
session = "M050_2025_01_17_14_00"
# session = "M047_2024_11_22_11_00"

animal = session[0:4]
data_dir = 'C:/Users/Cecilia/data/raw'
filename = session + '_pyaldata.mat'
df = prep.convert_mat_to_df(data_dir + '/' + animal + '/' + session + '/' + filename) # convert mat file to pandas dataframe
# df = prep.prep_general(df) # run general preprocessing

('animal', 'session', 'trial_id', 'trial_name', 'trial_length', 'bin_size', 'idx_trial_start', 'idx_trial_end', 'idx_motion', 'values_before_camera_trigger', 'idx_before_camera_trigger', 'values_CPI', 'idx_CPI', 'idx_cursor_update', 'values_reward_number', 'idx_reward_number', 'idx_lick', 'idx_session_timer', 'motion_sensor_xy', 'all_chan_best', 'all_unit_guide', 'all_KSLabel', 'all_spikes')


In [4]:
def load_BCI_log(file_path):
    # Assuming you already have the file content in a variable called `lines`
    with open(file_path, "r") as file:
        lines = file.readlines()

    # Parse the content into a DataFrame
    data = []
    for line in lines:
        try:
            # Extract timestamp: text between the first ":" and "-"
            timestamp = line.split(":")[1].split("-")[0].strip()
            
            # Extract log level: text after the first "-" and before the next ":"
            log_level = line.split("-", 2)[2].split(":")[0].strip()
            
            # Extract message: text after the second ":"
            message = line.split(":", 2)[2].strip()
            
            # Append parsed data
            data.append([timestamp, log_level, message])
        except (IndexError, ValueError):
            # Skip lines that don't match the expected format
            continue
    
    # Create the DataFrame
    df = pd.DataFrame(data, columns=["Timestamp", "LogLevel", "Message"])
    
    return(df)

In [5]:
# Load BCI txts
# Define the directory and session
data_dir = 'C:/Users/Cecilia/data/raw'
session = "M050_2025_01_17_14_00"
animal = session[0:4]

last_node = load_BCI_log(data_dir + '/' + animal + '/' + session +  '/' + session + '_Process-get_last_node.log')
middle_node = load_BCI_log(data_dir + '/' + animal + '/' + session +  '/' + session + '_Process-get_middle_node1.log')
save_node = load_BCI_log(data_dir + '/' + animal + '/' + session +  '/' + session + '_Process-get_save_node.log')
main = load_BCI_log(data_dir + '/' + animal + '/' + session +  '/' + session + '_Process-Main.log') #  THIS IS WRONG#
update_buffer = load_BCI_log(data_dir + '/' + animal + '/' + session +  '/' + session + '_Process-update_buffer.log')
update_pca = load_BCI_log(data_dir + '/' + animal + '/' + session +  '/' + session + '_Process-update_pca.log')

# Display the DataFrame
print(last_node.head())


                      Timestamp LogLevel  \
0  2025_01_17_14_19_50_0.068403    DEBUG   
1  2025_01_17_14_19_51_1.056383    DEBUG   
2  2025_01_17_14_19_51_1.203805     INFO   
3  2025_01_17_14_19_51_1.235594    DEBUG   
4  2025_01_17_14_19_51_1.236236  WARNING   

                                             Message  
0          Process-get_last_node logger initialised.  
1                            last node process start  
2                                      success: COM4  
3  serial ports: Serial<id=0x262f1dc35e0, open=Tr...  
4                    stats:canoncorr:NotFullRank = X  


In [6]:
display(last_node)

,Timestamp,LogLevel,Message
0,2025_01_17_14_19_50_0.068403,DEBUG,Process-get_last_node logger initialised.
1,2025_01_17_14_19_51_1.056383,DEBUG,last node process start
2,2025_01_17_14_19_51_1.203805,INFO,success: COM4
3,2025_01_17_14_19_51_1.235594,DEBUG,"serial ports: Serial<id=0x262f1dc35e0, open=Tr..."
4,2025_01_17_14_19_51_1.236236,WARNING,stats:canoncorr:NotFullRank = X
...,...,...,...
181147,2025_01_17_15_20_14_3623.748266,DEBUG,rolling cca sum=4.87267650100861 ccs=[0.800888...
181148,2025_01_17_15_20_14_3623.748466,DEBUG,cursor value not changed:0
181149,2025_01_17_15_20_14_3623.788551,DEBUG,rolling cca sum=4.928899548422445 ccs=[0.78901...
181150,2025_01_17_15_20_14_3623.788682,DEBUG,cursor value not changed:0
